# Earthquake dataset

**We will create a machine learning model to predict the magnitude of an earthquake**

#### Importing the required libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

#### Reading the dataset

In [ ]:
df = pd.read_csv('Earthquake.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

*The dataset has 9 categorical columns and 12 numerical columns. Some columns have missing values. *The shape of the dataframe shows that there 23412 rows in the dataset and 21 columns/features**

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

*There are high number of missing values for the columns Depth Error, Depth Seismic Stations, Magnitude Error, Magnitude Seismic Stations, Azimuthal Gap, Horizontal Distance and Horizontal Error. We can drop these columns as they large number of missing values. For Root Mean Square column, we can replace the missing values with mean/median value*

In [ ]:
df = df.drop(['Magnitude Error', 'Magnitude Seismic Stations', 'Horizontal Distance', 'Horizontal Error'], axis=1)

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum()

*We have dropped the colunms having huge number of missing values*

In [ ]:
df = df.dropna(subset=['Magnitude Type'])

In [ ]:
df.isnull().sum()

*We have dropped the rows corresponding to the 3 missing values in the column Magnitude Type. For the remaining columns with missing values, we will impute the missing values with mean/median*

#### Now we will check for duplicates

In [ ]:
df.duplicated().sum()

*There are no duplicate rows*

#### Checking for unique values

In [ ]:
df.nunique()

*The ID column has all unique values and thus it does not contribute to determine the magnitude of the earthquake. So we can drop the ID column*

In [ ]:
df = df.drop(['ID'], axis = 1)

In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

*The ID column has been dropped*

In [ ]:
NaN_features = [feature for feature in df.columns if ((df[feature].dtypes != 'O') & (df[feature].count() < df.shape[0]))]
NaN_features

In [ ]:
for feature in NaN_features:
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    sns.distplot(df[feature], color='red')
    plt.subplot(1,2,2)
    plt.hist(df[feature])

*These above plotted features have missing values. We need to impute the missing values in these columns. Since there is skewness in these features like **Depth Error**, **Depth seismic stations** and **Azimuthal gap**, we must use median to impute for these columns. for **Root mean square**, we can use mean to impute*

In [ ]:
df.describe()

In [ ]:
df['Depth Error'] = df['Depth Error'].fillna(df['Depth Error'].median())

In [ ]:
df['Depth Seismic Stations'] = df['Depth Seismic Stations'].fillna(df['Depth Seismic Stations'].median())

In [ ]:
df['Azimuthal Gap'] = df['Azimuthal Gap'].fillna(df['Azimuthal Gap'].median())

In [ ]:
df['Root Mean Square'] = df['Root Mean Square'].fillna(df['Root Mean Square'].mean())

In [ ]:
df.describe()

In [ ]:
for feature in NaN_features:
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    sns.distplot(df[feature], color='red')
    plt.subplot(1,2,2)
    plt.hist(df[feature])

In [ ]:
df.isnull().sum()

In [ ]:
df.head(10)

In [ ]:
df.info()

#### Plotting a heatmap to see the correlation between the numerical features

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df.corr(), annot=True)

*We can see that the features are weakly correlated to one another.* The pearson correlation of all the featurs is between -0.31 to 0.25*

In [ ]:
numerical_features = df.select_dtypes(np.number).columns
numerical_features

#### Plotting boxplots and distplots to see the outliers and skewness

In [ ]:

def create_boxplots_distplots(dataset):
    for feature in numerical_features:
        plt.figure(figsize=(10,7))
        plt.subplot(2,2,1)
        dataset.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.subplot(2,2,2)
        sns.boxplot(y=dataset[feature])
        plt.subplot(2,2,3)
        plt.hist(dataset[feature])
        plt.xlabel(feature)
        plt.subplot(2,2,4)
        sns.distplot(dataset[feature].dropna())
        plt.show()
    
create_boxplots_distplots(df)

#### Removing outliers

In [ ]:
def remove_outliers(dataset, list_of_features):
    for feature in list_of_features:
        outliers = []
        threshold = 3
        mean = np.mean(dataset[feature])
        std = np.std(dataset[feature])
        
        for i in dataset[feature]:
            z_score = (i - mean)/std
            if np.abs(z_score) > threshold:
                outliers.append(i)
        print(outliers)
        print()
        for j in outliers:
            indexNames = dataset[dataset[feature] == j].index
            dataset.drop(indexNames , inplace=True)
        outliers.clear()


remove_outliers(df,numerical_features)

In [ ]:
create_boxplots_distplots(df)

In [ ]:
df.describe()

*The outliers have been removed*

#### Now we will check for multivariate outliers by plotting relplot

In [ ]:
'''
def create_relplot(dataset, list_of_features):
    for feature in list_of_features:
        for bivariate_feature in list_of_features:
            if bivariate_feature != feature:
                plt.figure(figsize=(10,10))
                sns.relplot(x=feature, y=bivariate_feature, data=dataset, hue='Magnitude')
                plt.show()


create_relplot(df, numerical_features)
'''

In [ ]:
df = df.drop(df[(df['Depth'] > 225)].index)

In [ ]:
df = df.drop(df[(df['Latitude'] < -50)].index)

In [ ]:
df = df.drop(df[(df['Depth Error'] < 1)].index)

In [ ]:
df = df.drop(df[(df['Latitude'] > 60)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Depth Error'] > 4)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Depth Seismic Stations'] >300)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Depth Seismic Stations'] <200)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 150) & (df['Azimuthal Gap'] > 40)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Azimuthal Gap'] < 32)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Root Mean Square'] > 1.1)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Root Mean Square'] < 0.9)].index)

In [ ]:
df = df.drop(df[(df['Depth'] > 200) & (df['Latitude'] < -10)].index)

In [ ]:
#create_relplot(df, numerical_features)

*We have removed the multivariate outliers from the dataset*

In [ ]:
df.describe()

In [ ]:
create_boxplots_distplots(df)

*Features including **longitude**, **Depth** and **Magnitude** have skewness*

In [ ]:
df.head(10)

In [ ]:
df.describe()

#### Applying square root transformation to treat skewness

In [ ]:
from scipy.stats import skew

In [ ]:
for feature in numerical_features:
    print(feature, " ", skew(df[feature]))

In [ ]:
skewed_features = ['Depth', 'Depth Error', 'Azimuthal Gap']
for feature in skewed_features:
    df[feature] = np.sqrt(df[feature])

In [ ]:
for feature in numerical_features:
    print(feature, " ", skew(df[feature]))

*Skewness has been handled*

In [ ]:
df.head(10)

#### Analysing categorical features

In [ ]:
categorical_features = df.select_dtypes(np.object).columns
categorical_features

In [ ]:
for feature in categorical_features:
    if feature == 'Date':
        pass
    elif feature == 'Time':
        pass
    else:
        print("Feature : ", feature)
        print(df[feature].value_counts())
        print()

*We can consider the Nuclear explosion and Explosion as 1 single category*

In [ ]:
df["Type"].replace({"Nuclear Explosion": "Explosion"}, inplace=True)

*For the above features, we will remove the categories having counts less than 4*

In [ ]:
for feature in categorical_features:
    if feature == 'Date':
        pass
    elif feature == 'Time':
        pass
    elif feature == 'Location Source':
        pass
    else:
        x = df[feature].value_counts().keys().tolist()
        y = df[feature].value_counts().tolist()
        for i in range(len(x)):
            if y[i] < 4:
                index_names = df[df[feature] == x[i]].index
                df.drop(index_names, inplace = True)
        x = None
        y = None

In [ ]:
for feature in categorical_features:
    if feature == 'Date':
        pass
    elif feature == 'Time':
        pass
    else:
        print("Feature : ", feature)
        print(df[feature].value_counts())
        print()

*We can group the categories of **Location Source** column for which the count is less than 29*

In [ ]:
x = df['Location Source'].value_counts().keys().tolist()
y = df['Location Source'].value_counts().tolist()
for i in range(len(x)):
    if y[i] < 29:
        df['Location Source'].replace({x[i]: "Other"}, inplace=True)

In [ ]:
df['Location Source'].value_counts()

In [ ]:
for feature in categorical_features:
    if feature == 'Date':
        pass
    elif feature == 'Time':
        pass
    else:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        sns.countplot(feature, data=df)
        plt.title(feature)
        plt.subplot(1,2,2)
        plt.pie(df[feature].value_counts(), labels=df[feature].unique(), autopct='%0.2f%%')
        plt.show()
        sns.catplot(x=feature, y='Magnitude', data=df)
        plt.show()
        
        
        

*From the plots, we can observe that most of the earthquakes are genuine earthquakes whereas very few earthquakes are due to explosions. 99.24% of the earthquakes are genuine earthquakes whereas 0.76% earthquakes are due to explosions. the magnitude of earthquake ranges from 2.35 to approx 2.70 for genuine earthquakes whereas the magnitude varies from 2.35 to 2.55 in the case of explosions*

*Magnitude Type glossary :*

     MWC : centroid
     MWW : (Moment W-phase)(generic notation Mw)
     MB : short-period body wave
     MWB : body wave
     MS : 20 sec surface wave
     MWW : Moment W-phase)(generic notation Mw
     MWR : regional
     ML : local

#### Working with Date and Time values

In [ ]:
df.info()

*Date and Time columns are categorical features. We need to convert Date into datetime object*

In [ ]:
df['Day'] = pd.to_datetime(df.Date, format="%m/%d/%Y").dt.day

In [ ]:
df['Month'] = pd.to_datetime(df.Date, format="%m/%d/%Y").dt.month

In [ ]:
df['Year'] = pd.to_datetime(df.Date, format="%m/%d/%Y").dt.year

In [ ]:
df['Year'].value_counts()

*We can drop the Date column*

In [ ]:
df.drop(['Date'], axis=1, inplace=True)

In [ ]:
df['Hours'] = pd.to_datetime(df['Time']).dt.hour

In [ ]:
df['Minutes'] = pd.to_datetime(df['Time']).dt.minute

In [ ]:
df['Seconds'] = pd.to_datetime(df['Time']).dt.second

In [ ]:
df.drop(['Time'], axis=1, inplace=True)

In [ ]:
df.head(5)

In [ ]:
sns.jointplot(y=df['Magnitude'], x=df['Year'], data=df)

In [ ]:
plt.figure(figsize=(15,7))
sns.heatmap(df.corr(), annot=True, cmap='RdYlGn')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.to_csv('file6.csv')

In [39]:
df = pd.read_csv('file6.csv', index_col='Unnamed: 0')

#### Encoding of categorical variables

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(5)

In [ ]:
m = df.Magnitude
m

In [ ]:
df.drop(['Magnitude'], axis=1, inplace=True)

In [ ]:
df['Magnitude'] = m

In [ ]:
df.head(5)

#### Splitting the dataset

In [ ]:
X = df.iloc[:, :-1]

In [ ]:
y = df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=124)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

#### Model Building

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regressor = LinearRegression()

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
print("Intercept : ", regressor.intercept_)

In [ ]:
regressor.coef_

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
print("Mean absolute error : ", metrics.mean_absolute_error(y_test, y_pred))
print("Mean squared error : ", metrics.mean_squared_error(y_test, y_pred))
print("Root mean squared error : ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
y_pred

In [ ]:
sns.distplot(y_test-y_pred)

In [ ]:
y_test

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)

In [ ]:
#Number of features to consider at every split
max_features = ['auto', 'sqrt']
#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 5, stop = 30, num = 6)]
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
#Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

random_grid

In [ ]:
rfc = RandomForestRegressor()
rsc = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, scoring = 'neg_mean_squared_error', n_iter = 10, cv = 5, random_state = 42, verbose = 2, n_jobs=1)

In [ ]:
rsc.fit(X_train, y_train)

In [ ]:
prediction = rsc.predict(X_test)
prediction

In [ ]:
y_test

In [ ]:
rsc.best_params_

In [ ]:
plt.figure(figsize = (5,5))
sns.distplot(y_test-prediction)
plt.show()

In [ ]:
print("Mean absolute error : ", metrics.mean_absolute_error(y_test, prediction))
print("Mean squared error : ", metrics.mean_squared_error(y_test, prediction))
print("Root mean squared error : ", np.sqrt(metrics.mean_squared_error(y_test, prediction)))